# Day 3: Rucksack Reorganization
## Part 1

One Elf has the important job of loading all of the rucksacks with supplies for the jungle journey. Unfortunately, that Elf didn't quite follow the packing instructions, and so a few items now need to be rearranged.

Each rucksack has two large compartments. All items of a given type are meant to go into exactly one of the two compartments. The Elf that did the packing failed to follow this rule for exactly one item type per rucksack.

The Elves have made a list of all of the items currently in each rucksack (your puzzle input), but they need your help finding the errors. Every item type is identified by a single lowercase or uppercase letter (that is, a and A refer to different types of items).

The list of items for each rucksack is given as characters all on a single line. A given rucksack always has the same number of items in each of its two compartments, so the first half of the characters represent items in the first compartment, while the second half of the characters represent items in the second compartment.

For example, suppose you have the following list of contents from six rucksacks:

```
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
```

The first rucksack contains the items `vJrwpWtwJgWrhcsFMMfFFhFp`, which means its first compartment contains the items `vJrwpWtwJgWr`, while the second compartment contains the items `hcsFMMfFFhFp`. The only item type that appears in both compartments is lowercase p.
The second rucksack's compartments contain `jqHRNqRjqzjGDLGL` and `rsFMfFZSrLrFZsSL`. The only item type that appears in both compartments is uppercase L.
The third rucksack's compartments contain `PmmdzqPrV` and `vPwwTWBwg`; the only common item type is uppercase P.
The fourth rucksack's compartments only share item type v.
The fifth rucksack's compartments only share item type t.
The sixth rucksack's compartments only share item type s.
To help prioritize item rearrangement, every item type can be converted to a priority:

Lowercase item types a through z have priorities 1 through 26.
Uppercase item types A through Z have priorities 27 through 52.
In the above example, the priority of the item type that appears in both compartments of each rucksack is 16 (p), 38 (L), 42 (P), 22 (v), 20 (t), and 19 (s); the sum of these is 157.

Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?

In [17]:
import pandas as pd
from collections import Counter
import re

In [18]:
def value_cal(x: str):
    """Using the ord (unicode to int) calculate the priority mapping
    [a-z] = [ 1 -> 26]
    [A-Z] = [27 -> 52]
    """
    if x.islower():
        return ord(x) - 96
    return ord(x) - 38

In [47]:
# Read in the puzzle
part1 = pd.read_table("input.txt", header=None, dtype="string")
# Set an initial column name
part1.columns = ["Rucksack"]
# Get a count of the number of items by getting the length of each string
part1["Items"] = part1["Rucksack"].str.len()
# Calculate the halfway point which will be used for the boundary between compartments
part1["Midpoint"] = (part1["Items"] / 2).astype(int)
# For each row, using the midpoint value, slice off the left compartment
part1["Left"] = [ row.Rucksack[slice(0, row.Midpoint)] for row in part1.itertuples(index=False) ]
# For each row, using the midpoint value, slice off the right compartment
part1["Right"] = [ row.Rucksack[slice(row.Midpoint, row.Items)] for row in part1.itertuples(index=False) ]
# Using the power of set intersection find the common character in each string
part1["Common"] = [ str(*set.intersection(set(row.Left), set(row.Right))) for row in part1.itertuples(index=False) ]
# Use the value_cal function above to map the character to an int priority
part1["Value"] = part1["Common"].map(value_cal)
# Sum all the priorities
part1["Value"].sum()

8243

## Part Two
As you finish identifying the misplaced items, the Elves come to you with another issue.

For safety, the Elves are divided into groups of three. Every Elf carries a badge that identifies their group. For efficiency, within each group of three Elves, the badge is the only item type carried by all three Elves. That is, if a group's badge is item type B, then all three Elves will have item type B somewhere in their rucksack, and at most two of the Elves will be carrying any other item type.

The problem is that someone forgot to put this year's updated authenticity sticker on the badges. All of the badges need to be pulled out of the rucksacks so the new authenticity stickers can be attached.

Additionally, nobody wrote down which item type corresponds to each group's badges. The only way to tell which item type is the right one is by finding the one item type that is common between all three Elves in each group.

Every set of three lines in your list corresponds to a single group, but each group can have a different badge item type. So, in the above example, the first group's rucksacks are the first three lines:

```
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
```

And the second group's rucksacks are the next three lines:

```
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
```

In the first group, the only item type that appears in all three rucksacks is lowercase r; this must be their badges. In the second group, their badge item type must be Z.

Priorities for these items must still be found to organize the sticker attachment efforts: here, they are 18 (r) for the first group and 52 (Z) for the second group. The sum of these is 70.

Find the item type that corresponds to the badges of each three-Elf group. What is the sum of the priorities of those item types?

In [61]:
# Use a regex to extract three rucksacks at once
part2 = re.findall(r'(\w+\n\w+\n\w+)', open("input.txt", "r").read(), re.MULTILINE)
# Create a DataFrame for the group, then split by the newline character into individual columns
part2 = pd.DataFrame(part2, columns=["Group"], dtype="string")["Group"].str.split('\n', expand=True)
# Name the new columns
part2.columns = ["Elf1", "Elf2", "Elf3"]
# Using the power of set intersection find the common character in all the strings
part2["Badge"] = [ str(*set.intersection(set(row.Elf1), set(row.Elf2), set(row.Elf3))) for row in part2.itertuples(index=False) ]
# Use the value_cal function above to map the character to an int priority
part2["Value"] = part2["Badge"].map(value_cal)
# Sum all the priorities 
part2["Value"].sum()

2631